In [37]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from email import policy
from email.parser import BytesParser
import jieba

## 使用email库提取邮件内容
1. 读取eml文件
2. 提取邮件头
3. 提取邮件正文（不包括附件），注意到邮件正文可能是html格式或base64编码格式（图片），使用bs4库进行解析
4. 将邮件数据保存为DataFrame

In [38]:
def get_body(message):
    if message.is_multipart():
        return get_body(message.get_payload(0))
    else:
        return message.get_payload(None, True)

def eml_to_dict(eml_path):
    with open(eml_path, 'rb') as f:
        msg = BytesParser(policy=policy.default).parse(f)
    # get headers
    data = dict(msg.items())
    # get body
    body = get_body(msg)
    if body is not None:
        try:
            data['body'] = body.get_content()
        except AttributeError:
            data['body'] = body
        # get text from html
        data['body'] = BeautifulSoup(data['body'], "html.parser").text.strip()
    data['email_id'] = os.path.basename(eml_path).split('.')[0]
    return data

# dataset path
eml_dir = "D:\\notebook\datacon\dataset"

eml_files = [os.path.join(eml_dir, filename) for filename in os.listdir(eml_dir) if filename.endswith(".eml")]

data = [eml_to_dict(eml_file) for eml_file in eml_files]
df = pd.DataFrame(data)

C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\721142775.py:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['body'] = BeautifulSoup(data['body'], "lxml").text.strip()


## 查看数据集
- 4277条数据，包含630个字段
- 分析得知，邮件正文的内容在body字段中（若为空值通常表示正文为图片或其余非html格式）
- 邮件头中的Subject字段包含邮件主题
- 其余邮件头多数为空值或经过脱敏处理

In [39]:
df

,Received,X-Securemailgate-Identity,Message-Id,Subject,Date,X-Priority,X-Mailer,Mime-Version,Content-Type,X-Originating-Ip,...,X-Rm-Spam,X-Hqip,X-Sfdc-Lk,X-Sfdc-User,X-Mail_abuse_inquiries,X-Sfdc-Tls-Norelay,X-Sfdc-Binding,X-Sfdc-Emailcategory,X-Sfdc-Entityid,X-Sfdc-Interface
0,from xsaypzzai (unknown [180.106.88.161])\t(us...,dreyero_de;web277.dogado.net,<936e72d64105baa9a1dbe2640e6f5d81@dreyero.de>,2023年第一季度《财 政》补〉贴,"Mon, 13 Feb 2023 10:01:30 +0800",3,Cvzlvliqe Wydootqd 37.9,1.0,"multipart/related; boundary=""44b28501923df81c8...",31.47.255.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,from mail.nfqwao.top (mail.nfqwao.top [94.131....,NaN,<684317841.2471931.1676254079269@mail.nfqwao.top>,三八妇女节的礼品，准备好了吗？,"Mon, 13 Feb 2023 10:07:59 +0800",NaN,NaN,1.0,"multipart/mixed; boundary=""----=_Part_2471930_...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,from zbyvlxgzm (unknown [49.85.250.111])\tby m...,NaN,<91E13A01FF5D908B806E8253D52BCD1E@zbyvlxgzm>,邮箱系统在线升级,"Tue, 22 Aug 2023 01:33:46 +0800",3,Microsoft Outlook Express 6.00.2900.5512,1.0,"multipart/alternative; boundary=""----=_NextPar...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,高薪工作提拔晋升户口出国轻松全搞定tianjinAD,"Mon, 13 Feb 2023 11:05:30 +0800",2,EhooPost 2004b,NaN,"text/plain; charset=""GB2312""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,from josie (unknown [123.11.53.20])\t(Authenti...,NaN,<63E9AA25.1B4E5E.14145@qn-cmmxproxy-4>,转发： 刘建华老师 收,"Mon, 13 Feb 2023 11:04:00 +0800",NaN,NaN,1.0,"multipart/mixed; charset=""UTF-8""; boundary=""sa...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,from mail.cciy.top (mail.cciy.top [193.201.126...,NaN,<1728955067.808585.1676254668606@mail.cciy.top>,三八妇女节！贵司完成礼品采购了吗？,"Mon, 13 Feb 2023 10:17:48 +0800",NaN,NaN,1.0,"multipart/mixed; boundary=""----=_Part_808584_3...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4273,from [115.219.6.56] (port=51513 helo=jutchefkw...,NaN,<9d2eb98d929ff192aa4aa29a35fac9ef@sistemasimpl...,个人劳动（补贴））已下发，请查看下图查收,"Mon, 13 Feb 2023 10:01:06 +0800",3,Ohbtef Bukgybuiiu 8.94,1.0,"multipart/related; boundary=""7cddf233ae28b4399...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4274,from tbq (unknown [180.108.236.38])\tby mail.i...,NaN,<1FA8A33A5750EC0F102DAFC7825CDE4C@tbq>,邮箱系统在线升级,"Tue, 22 Aug 2023 13:05:50 +0800",3,Microsoft Outlook Express 6.00.2900.5512,1.0,"multipart/alternative; boundary=""----=_NextPar...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4275,from eeydxfmjnl (unknown [122.230.145.252])\tb...,NaN,<5adc13a1db92fd10ed1eceb60dc7180c@luckymoonpac...,2023年第一季度《财 政》补〉贴,"Mon, 13 Feb 2023 11:06:20 +0800",3,Jxczs Ohnhwu 12.78,1.0,"multipart/related; boundary=""3978b51d8dea23bcf...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.shape

(4277, 630)

## 特征提取

- 提取邮件主题(Subject)和正文(body)内容
- 提取邮件名的前八位，方便具体分析
- Received 属性可能包含特征信息，但分析难度较大，故放弃  

In [43]:
content = df[['email_id','Subject', 'body']]

In [44]:
content

,email_id,Subject,body
0,000249d6,2023年第一季度《财 政》补〉贴,
1,0016a8f1,三八妇女节的礼品，准备好了吗？,Hello，你好1、我是为企业提供礼品采购的陈晓飞，如：商务伴手礼、节日礼品、活动礼品、员工...
2,003804af,邮箱系统在线升级,亲爱的用户： \r\n为了加强网络安全管理，提高邮件系统的安全性和稳定性，保障收发畅通，为用...
3,0040c33b,高薪工作提拔晋升户口出国轻松全搞定tianjinAD,"历经三年疫情洗礼，实业蓄能已久，人工智能AI技术WEB3.0, 区块链技术应用场景会越来越广..."
4,00493728,转发： 刘建华老师 收,您好！之前给您发过关于2月底将在线上开展 “表观遗传学-DNA甲基化 Rna甲基化 M6a甲...
...,...,...,...
4272,ffc67ed7,三八妇女节！贵司完成礼品采购了吗？,Hello，你好﻿1、我是陈晓飞 Felix，为企业提供礼品采购的企业：商务馈赠、节日礼品、...
4273,ffcc00a4,个人劳动（补贴））已下发，请查看下图查收,
4274,ffce9394,邮箱系统在线升级,亲爱的用户： \r\n为了加强网络安全管理，提高邮件系统的安全性和稳定性，保障收发畅通，为用...
4275,ffd7b406,2023年第一季度《财 政》补〉贴,


## 特征分析
- 注意到邮件主题中包含了大量的空格、换行符、制表符等，需要进行清洗
- 注意到4277封邮件中只有878个独立主题，需要进行去重


In [45]:
content.describe()

,email_id,Subject,body
count,4277,4277,4277
unique,4277,878,1116
top,000249d6,邮箱系统在线升级,
freq,1,618,929


In [46]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   email_id  4277 non-null   object
 1   Subject   4277 non-null   object
 2   body      4277 non-null   object
dtypes: object(3)
memory usage: 100.4+ KB


## 数据清洗
- 去除空格、换行符、制表符等
- 去除非UTF-8编码的字符
- 丢弃清洗后主题为空的邮件
- 去除重复邮件

In [47]:
content['Subject'] = content['Subject'].str.replace(' ', '').str.replace('〉', '').str.replace('\n', '').str.replace('\r', '').str.replace('\t', '').str.replace('\xa0', ' ').str.replace('\ufeff', ' ').str.replace('◖','')
content['body'] = content['body'].str.replace('\n', '').str.replace('\r', '').str.replace('\t', '').str.replace('\xa0', ' ').str.replace('\ufeff', ' ')
content.drop(content.loc[content['Subject'] == ''].index, inplace=True)
content.drop_duplicates(subset=['Subject'], keep='first', inplace=True)

C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\3541818220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['Subject'] = content['Subject'].str.replace(' ', '').str.replace('〉', '').str.replace('\n', '').str.replace('\r', '').str.replace('\t', '').str.replace('\xa0', ' ').str.replace('\ufeff', ' ').str.replace('◖','')
C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\3541818220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['body'] = content['body'].str.replace('\n', '').str.replace('\

## 对清洗去除后的数据集分析
- 仅剩下863条数据
- 如第1条email（2023年第一季度《财政》补贴），正文内容为空，打开邮件观察得知，该邮件正文为图片，故正文为能正常解析，且为冒充政府机构的**钓鱼邮件**
- 如第3条email（邮箱系统在线升级），这一类邮件在之前分析中存在大量重复，打开邮件观察得知，该类邮件属于冒充邮箱管理员的**钓鱼邮件**

In [48]:
content

,email_id,Subject,body
0,000249d6,2023年第一季度《财政》补贴,
1,0016a8f1,三八妇女节的礼品，准备好了吗？,Hello，你好1、我是为企业提供礼品采购的陈晓飞，如：商务伴手礼、节日礼品、活动礼品、员工...
2,003804af,邮箱系统在线升级,亲爱的用户： 为了加强网络安全管理，提高邮件系统的安全性和稳定性，保障收发畅通，为用户提供优...
3,0040c33b,高薪工作提拔晋升户口出国轻松全搞定tianjinAD,"历经三年疫情洗礼，实业蓄能已久，人工智能AI技术WEB3.0, 区块链技术应用场景会越来越广..."
4,00493728,转发：刘建华老师收,您好！之前给您发过关于2月底将在线上开展 “表观遗传学-DNA甲基化 Rna甲基化 M6a甲...
...,...,...,...
4243,fe2242b6,中国智造企业全球化人才吸引与发展新引擎—领英智能制造行业线上研讨会，期待您的免费上线观看！,智能制造行业峰会 人才“智造”出海机遇赢在出海在线浏览请点击[1] 这里[2]智能制造行业峰...
4253,fee2719a,DHLSHIPPINGDOCUMENT,">>>>>>>>>>>>Dear Customer, >>>> >>>>There is a..."
4254,fef188cd,RE:914790529、914704622帮忙通知厦门外代出具改单授权委托书[ref:_0...,"Dear 外代以下客户要求请协助帮忙，谢谢Regards,Derreck WuReprese..."
4266,ff753c51,報價請求：重慶INV212AS//114CN,尊敬的先生/女士，請發送更新的價格列表，以進行新的新要求。 我們對購買非常感興趣。 如果您可...


## 自动化标注
- 通过观察，发现钓鱼邮件的特征为：邮件主题中包含“薪资、工资、财政、补助、补贴、津贴、邮箱、邮件、及时、系统、通告、警告、抵扣、增值、税务”等词汇，正文中包含“郵箱、系統、money”等词汇
- 通过正则表达式进行匹配，将钓鱼邮件标记为1，其余邮件标记为0
- 新增属性class，表示对此封邮件的分类标注


In [49]:
a = content['Subject'].apply(lambda x: 1 if any(word in x for word in ['薪资','工资','财政','补助','补贴','津贴','邮箱','邮件','及时','系统','通告', '警告','抵扣','增值','税务']) else 0)
b =content['body'].apply(lambda x: 1 if any(word in x for word in ['郵箱','系統','money']) else 0)
content['class'] = a | b

C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\2815912577.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['class'] = a | b


In [50]:
content['class'].value_counts()

0    622
1    241
Name: class, dtype: int64

## 空值处理
- 由于部分正文内容为空，这里采用一种简单的空值处理办法
- 将主题与正文内容合并为Message
- 提取Message和class两列，作为需要学习的数据

In [51]:
content['Message'] = content['Subject'] + ' ' + content['body']

C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\2827404937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['Message'] = content['Subject'] + ' ' + content['body']


In [52]:
data = content[['Message', 'class']]

In [53]:
data.to_csv('D:\\notebook\datacon\data.csv', index=False)
data

,Message,class
0,2023年第一季度《财政》补贴,1
1,三八妇女节的礼品，准备好了吗？ Hello，你好1、我是为企业提供礼品采购的陈晓飞，如：商务...,0
2,邮箱系统在线升级 亲爱的用户： 为了加强网络安全管理，提高邮件系统的安全性和稳定性，保障收发...,1
3,高薪工作提拔晋升户口出国轻松全搞定tianjinAD 历经三年疫情洗礼，实业蓄能已久，人工智...,0
4,转发：刘建华老师收 您好！之前给您发过关于2月底将在线上开展 “表观遗传学-DNA甲基化 R...,0
...,...,...
4243,中国智造企业全球化人才吸引与发展新引擎—领英智能制造行业线上研讨会，期待您的免费上线观看！ ...,0
4253,"DHLSHIPPINGDOCUMENT >>>>>>>>>>>>Dear Customer,...",0
4254,RE:914790529、914704622帮忙通知厦门外代出具改单授权委托书[ref:_0...,0
4266,報價請求：重慶INV212AS//114CN 尊敬的先生/女士，請發送更新的價格列表，以進行...,0


## 分词
- 英文天然具有分词属性，中文则需要进行分词
- 由于邮件主题和正文内容绝大数为中文，这里采用jieba库进行分词
- 分词有助于特征提取，提高分类准确率

In [62]:
data['Message'] = data['Message'].apply(lambda x: ' '.join(jieba.cut(x)))

C:\Users\Shen\AppData\Local\Temp\ipykernel_4452\2702886797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Message'] = data['Message'].apply(lambda x: ' '.join(jieba.cut(x)))


In [63]:
data

,Message,class
0,2023 年 第一季度 《 财政 》 补贴,1
1,三八妇女节 的 礼品 ， 准备 好了吗 ？ Hello ， 你好 1 、 我 是 为 企...,0
2,邮箱 系统 在线 升级 亲爱 的 用户 ： 为了 加强 网络安全 管理 ， 提高 邮...,1
3,高薪 工作 提拔 晋升 户口 出国 轻松 全 搞定 tianjinAD 历经 三年 疫情...,0
4,转发 ： 刘建华 老师 收 您好 ！ 之前 给 您 发过 关于 2 月底 将 在线 上 ...,0
...,...,...
4243,中国 智造 企业 全球化 人才 吸引 与 发展 新 引擎 — 领英 智能 制造 行业 线上 ...,0
4253,DHLSHIPPINGDOCUMENT > > > > > > > > > > > > ...,0
4254,RE : 914790529 、 914704622 帮忙 通知 厦门 外代 出具 改单 授...,0
4266,報價 請求 ： 重慶 INV212AS / / 114CN 尊敬 的 先生 / 女士 ，...,0


## 模型设计
- 使用sklearn库进行模型设计以及数据集划分
- 使用Pipeline进行数据预处理（特征选择、标准化、PCA等）
- 使用CountVectorizer对分词进行频率统计（词袋模型）
- 使用MultinomialNB对词袋模型特征向量化并基于多项式朴素贝叶斯算法进行分类


### 基本思想
1. 学习不同分类的邮件的特征
2. 基于特征出现的概率，对邮件进行分类，如：某封邮件的Message若同时出现“邮箱”、“系统”、“升级”、“密码”等特征词时，该邮件属于钓鱼邮件的概率更大
3. 朴素贝叶斯分类器的基本思想是：假设特征之间相互独立，即某封邮件的Message若同时出现“邮箱”、“系统”、“升级”、“密码”等特征词时，这些特征词之间相互独立，不会相互影响，但这也缺乏对整体邮件内容的相关性分析

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['Message'],data['class'],test_size=0.25)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [58]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

## 模型验证
- 对单独数据的验证
- 对测试集的验证，准确率为98%

In [59]:
emails=['''CraftBeerChina2023|亞洲國際精釀啤酒會議暨展覽會|2023.5.30~6.1|14.2億的人口市場等您來開發|疫後商機等您來開發!!!'''
    ,'''你好，本公司有增值税（票），代开全国各地类发票发票真实有效，网上可验证，点数优惠，绝对保真可线下取票/邮寄取票，联系徽信：131-2259-9416（电)  sh在仍处于阴影中的山谷草地上卵酣炮除朵锋摇寅炎厘眉支征低化箕sh@peony.cn'''
    ,'''邮箱OA更新各位公司同事，您好！正在OA系统更新,没有进行OA系统更新的用户三天后将无法使用。您当前的IMAP版本较低，建议立即升级点击进行OA更新小贴士：IMAP属于同步协议，您在移动端或MAC端需要正常使用，请在2023-2-20前完成OA系统更新！SESSIONID:MentAQAAAGaX6WMFLAAAIP:59.125.11.37ClientPort:25HASIPINFO:0...''',
    '''E-mail郵箱系統計劃於即日起開始進行遷移升級請如實填寫下列信息，回復此郵件！姓名：職位：郵箱賬號：登陸密碼：歷史密碼：註：請在收到郵件後立即進行回復，逾時將被回收賬號！d9rnfdh2   কমিশনের অফিস সহকারী/ডাটা এন্ট্রি অপারেটর জনাব মোঃ তহিদুল ইসলাম এর পাসপোর্ট অনাপত্তি সনদকমিশনের অফিস...
    ''',
    '''
    2023年第一季度《财政》补贴
    ''',
    '''高薪工作提拔晋升户口出国轻松全搞定tianjinAD 历经三年疫情洗礼，实业蓄能已久，人工智能AI技术WEB3.0, 区块链技术应用场景会越来越广泛，作为各类大企业需要新型人才的若你，将更加需要专业高端学历的加持，增强核心竞争力的砝码！如果你有能力缺平台 ，那么我们可以让你实现职场质的跨越！学信网全网独家的学历学位大数据融合平台（BDP  big  data platform  ,data...''',
    '''做账抵扣冲账'''
]
emails = [' '.join(jieba.cut(email)) for email in emails]

In [60]:
clf.predict(emails)

array([0, 1, 0, 1, 1, 0, 0], dtype=int64)

In [61]:
clf.score(X_test,y_test)

0.9351851851851852

## 查看测试集预测结果

In [34]:
y_pred = clf.predict(X_test)

# Create a DataFrame with actual and predicted values
results = pd.DataFrame({
    'Message': X_test,
    'Actual': y_test,
    'Predicted': y_pred
})
results

,Message,Actual,Predicted
510,回复 ： 流程 变革 （ 2 月 23 - 26 日 深圳 ） 您好 ！ ...,0,0
397,测量 技术 的 趋势 与 创新 View online version ...,0,0
1505,2023 - 02 - 13 日 通告 ： qwsun,1,1
3156,2023 - 02 - 13 日 通告 ： huangqiongrui,1,1
672,2023 - 02 - 13 日 通告 ： songxinjie,1,1
...,...,...,...
493,RE ： Untitled document 您好 ， 抱歉 打扰 了 。 如果 您...,0,0
2249,2023 - 02 - 13 日 通告 ： huanghui5,1,1
1895,2023 - 02 - 13 日 通告 ： 13110240014,1,1
1518,2023 - 02 - 13 日 通告 ： xfli,1,1


## 模型保存

In [36]:
from joblib import dump
dump(clf, 'email_predict.joblib') 

['email_predict.joblib']